Rusty Bargain used car sales service is developing an app to attract new customers. In that app, you can quickly find out the market value of your car. You have access to historical data: technical specifications, trim versions, and prices. You need to build the model to determine the value. 

Rusty Bargain is interested in:

- the quality of the prediction;
- the speed of the prediction;
- the time required for training

# 1. Data preparation

In [ ]:
#Imports

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score


In [ ]:
#reading data

data=pd.read_csv('/datasets/car_data.csv')
data.head()

#Dropped uniformative columns

data1=data.drop(['DateCrawled', 'DateCreated', 'LastSeen', 'Model','NumberOfPictures'], axis=1)
data1.info()
print(data1.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 11 columns):
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Mileage              354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
PostalCode           354369 non-null int64
dtypes: int64(6), object(5)
memory usage: 29.7+ MB
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Mileage                  0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
PostalCode               0
dtype: int64


The droped columns have too many unique values which will increase the size of features during OHE, hence these columns are dropped.

In [ ]:
#filling NA values

data1['VehicleType']=data1['VehicleType'].fillna(method='ffill')
data1['Gearbox']=data1['Gearbox'].fillna(method='ffill')
data1['FuelType']=data1['FuelType'].fillna(method='ffill')
data1['NotRepaired']=data1['NotRepaired'].fillna(method='ffill')
print(data1.isnull().sum())
data1.dropna(inplace=True)
data1.reset_index()
data1.info()

Price                0
VehicleType          1
RegistrationYear     0
Gearbox              0
Power                0
Mileage              0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          1
PostalCode           0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 354368 entries, 1 to 354368
Data columns (total 11 columns):
Price                354368 non-null int64
VehicleType          354368 non-null object
RegistrationYear     354368 non-null int64
Gearbox              354368 non-null object
Power                354368 non-null int64
Mileage              354368 non-null int64
RegistrationMonth    354368 non-null int64
FuelType             354368 non-null object
Brand                354368 non-null object
NotRepaired          354368 non-null object
PostalCode           354368 non-null int64
dtypes: int64(6), object(5)
memory usage: 32.4+ MB


In [ ]:
#OHE for categorical values

data_ohe=pd.get_dummies(data1, drop_first=True)
print(data_ohe.shape)
print(data_ohe.columns)

(354368, 60)
Index(['Price', 'RegistrationYear', 'Power', 'Mileage', 'RegistrationMonth',
       'PostalCode', 'VehicleType_convertible', 'VehicleType_coupe',
       'VehicleType_other', 'VehicleType_sedan', 'VehicleType_small',
       'VehicleType_suv', 'VehicleType_wagon', 'Gearbox_manual',
       'FuelType_electric', 'FuelType_gasoline', 'FuelType_hybrid',
       'FuelType_lpg', 'FuelType_other', 'FuelType_petrol', 'Brand_audi',
       'Brand_bmw', 'Brand_chevrolet', 'Brand_chrysler', 'Brand_citroen',
       'Brand_dacia', 'Brand_daewoo', 'Brand_daihatsu', 'Brand_fiat',
       'Brand_ford', 'Brand_honda', 'Brand_hyundai', 'Brand_jaguar',
       'Brand_jeep', 'Brand_kia', 'Brand_lada', 'Brand_lancia',
       'Brand_land_rover', 'Brand_mazda', 'Brand_mercedes_benz', 'Brand_mini',
       'Brand_mitsubishi', 'Brand_nissan', 'Brand_opel', 'Brand_peugeot',
       'Brand_porsche', 'Brand_renault', 'Brand_rover', 'Brand_saab',
       'Brand_seat', 'Brand_skoda', 'Brand_smart', 'Brand_sonsti

In [ ]:
#Spliting data into target and validation set and standarising data

target=data_ohe['Price']
features=data_ohe.drop('Price', axis=1)
features_train, features_test, target_train, target_test=train_test_split(features, target, test_size=0.25, random_state=12345)

numeric=['RegistrationYear', 'Power', 'Mileage', 'RegistrationMonth','PostalCode']

scaler=StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric]=scaler.transform(features_train[numeric])
features_test[numeric]=scaler.transform(features_test[numeric])

print(features_train.shape)
print(features_test.shape)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


(265776, 59)
(88592, 59)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


# 2. Model training

In [ ]:
#Linear regression

model_LR=LinearRegression()
%time model_LR.fit(features_train, target_train)


%time pred_LR=model_LR.predict(features_test)
rmse_LR=mean_squared_error(target_test,pred_LR)**0.5
print("Linear Regression RMSE:",rmse_LR)


CPU times: user 2.4 s, sys: 1.24 s, total: 3.63 s
Wall time: 3.61 s
CPU times: user 31.4 ms, sys: 61.3 ms, total: 92.7 ms
Wall time: 122 ms
Linear Regression RMSE: 3484.852558653384


In [ ]:
%%time
#hyperparameter tuning process for the random forest

model_RF=RandomForestRegressor()
param_RF={'n_estimators':[80, 90, 100],'max_depth':[30,40,50]}
gd_sr_RF=GridSearchCV(estimator=model_RF, param_grid = param_RF, cv = 3, n_jobs = -1, verbose = 2)
gd_sr_RF.fit(features_train, target_train)
gd_sr_RF.best_params_

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] max_depth=30, n_estimators=60 ...................................


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .................... max_depth=30, n_estimators=60, total= 1.8min
[CV] max_depth=30, n_estimators=60 ...................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.8min remaining:    0.0s


[CV] .................... max_depth=30, n_estimators=60, total= 1.8min
[CV] max_depth=30, n_estimators=60 ...................................
[CV] .................... max_depth=30, n_estimators=60, total= 2.0min
[CV] max_depth=30, n_estimators=80 ...................................
[CV] .................... max_depth=30, n_estimators=80, total= 2.4min
[CV] max_depth=30, n_estimators=80 ...................................
[CV] .................... max_depth=30, n_estimators=80, total= 2.5min
[CV] max_depth=30, n_estimators=80 ...................................
[CV] .................... max_depth=30, n_estimators=80, total= 2.6min
[CV] max_depth=40, n_estimators=60 ...................................
[CV] .................... max_depth=40, n_estimators=60, total= 1.9min
[CV] max_depth=40, n_estimators=60 ...................................
[CV] .................... max_depth=40, n_estimators=60, total= 1.9min
[CV] max_depth=40, n_estimators=60 ...................................
[CV] .

[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 41.7min finished


CPU times: user 46min 1s, sys: 1.27 s, total: 46min 2s
Wall time: 46min 10s


{'max_depth': 40, 'n_estimators': 80}

In [ ]:
model_RF_test=RandomForestRegressor(max_depth=40, n_estimators=80, random_state=12345)
%time model_RF_test.fit(features_train, target_train)

%time model_RF_pred=model_RF_test.predict(features_test)
rmse_RF=mean_squared_error(target_test, model_RF_pred)**0.5
print('rmse_RF:', rmse_RF)

CPU times: user 4min 31s, sys: 819 ms, total: 4min 32s
Wall time: 4min 33s
CPU times: user 5.08 s, sys: 7.43 ms, total: 5.08 s
Wall time: 5.09 s
rmse_RF: 1779.4234940368472


GridSearchCV shows best parameters as max_depth=40 and n_estimators=80

In [ ]:
#lightGBM

#changing data types to category for lightGBM
data1['VehicleType']=data1['VehicleType'].astype('category')
data1['Gearbox']=data1['Gearbox'].astype('category')
data1['FuelType']=data1['FuelType'].astype('category')
data1['NotRepaired']=data1['NotRepaired'].astype('category')
data1['Brand']=data1['Brand'].astype('category')

#Spliting data into target and test set
tar=data1['Price']
feat=data1.drop('Price', axis=1)
feat_train, feat_test, tar_train, tar_test=train_test_split(feat, tar, test_size=0.25, random_state=12345)

model_lgb=LGBMRegressor(num_leaves=50, max_depth=6, learning_rate=0.2, verbose=10, feature_fraction=0.6)
%time model_lgb.fit(feat_train, tar_train)

%time pred_lgb=model_lgb.predict(feat_test)
rmse_lgb=mean_squared_error(tar_test, pred_lgb)**0.5
print('rmse_lgb:', rmse_lgb)


CPU times: user 5min 21s, sys: 1.53 s, total: 5min 23s
Wall time: 5min 24s
CPU times: user 1.45 s, sys: 0 ns, total: 1.45 s
Wall time: 1.5 s
rmse_lgb: 1845.6702603415713


In [ ]:
#catboost

cat_features=['VehicleType', 'Gearbox', 'FuelType', 'NotRepaired', 'Brand']

model_cat = CatBoostRegressor(iterations=50, learning_rate=0.5)
%time model_cat.fit(feat_train, tar_train, cat_features=cat_features, verbose=10)

%time pred_cat=model_cat.predict(feat_test)
rmse_cat=mean_squared_error(tar_test, pred_lgb)**0.5
print('rmse_cat:', rmse_cat)

0:	learn: 3299.2275274	total: 482ms	remaining: 23.6s
10:	learn: 2088.6686588	total: 4.27s	remaining: 15.1s
20:	learn: 2007.4571125	total: 7.67s	remaining: 10.6s
30:	learn: 1962.6346228	total: 11.2s	remaining: 6.83s
40:	learn: 1928.3818079	total: 14.7s	remaining: 3.22s
49:	learn: 1907.8837838	total: 17.8s	remaining: 0us
CPU times: user 15.3 s, sys: 3.41 s, total: 18.7 s
Wall time: 20 s
CPU times: user 140 ms, sys: 16.5 ms, total: 156 ms
Wall time: 139 ms
rmse_cat: 1845.6702603415713


Used GridSearch for hyperparameter tuning in randomForestregressor.
Tuned both max_depth and n_estimators for Randomforest
Calculated time for both training and prediction

# 3. Model analysis

In [ ]:
print("Linear Regression RMSE:",rmse_LR)
print('rmse_RF:', rmse_RF)
print('rmse_lgb:', rmse_lgb)
print('rmse_cat:', rmse_cat)

Linear Regression RMSE: 3484.852558653384
rmse_RF: 1779.4234940368472
rmse_lgb: 1845.6702603415713
rmse_cat: 1845.6702603415713


All three models, RandomForestRegressor, LightGBM and Catboost performed much better than Linear rgression (LR). The hypertuning time for RF through GridSearch took more than 46 min while training time was 4min. Training time for LightGBM and Catboost.

MSE was lowest for RF, but further fine tuning in the hyperparameters of LightGBM and Catboost can be done to further reduce the mse value.

Catboost has much lower training time then LightGBM while the mse is same for both.